### Importamos modulos

In [2]:
import numpy as np
from numpy import load
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop, SGD
import tensorflow as tf
from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns




Using TensorFlow backend.


###  Cargamos Datasets

In [5]:
test_imagenes  = np.load('./Input/test_imagenes.npy')
test_etiqueta  = np.load('./Input/test_etiqueta.npy')
etiqueta_fruta = np.load('./Input/etiqueta_fruta.npy')
imagen_fruta   = np.load('./Input/imagen_fruta.npy')
nombre_frutas_corregido   = pd.read_csv("./Input/nombre_Frutas.csv")

### Organizamos los dataset en entrenamiento y pruebas

In [6]:
X_train, X_test = imagen_fruta, test_imagenes
y_train, y_test = etiqueta_fruta, test_etiqueta


#Hago una versión aplanada para algunos de nuestros modelos.
X_flat_train = X_train.reshape(X_train.shape[0], 35*35*3)
X_flat_test = X_test.reshape(X_test.shape[0], 35*35*3)

X_flat_train = X_flat_train/255
X_flat_test = X_flat_test/255 
#Categorizamos los valores de salida en un array de 50
y_train = keras.utils.to_categorical(y_train,num_classes =63)
y_test = keras.utils.to_categorical(y_test,num_classes =63)

#d2_train_dataset = train_dataset.reshape((nsamples,nx*ny))
print('Tamaños originales:', X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print('Tamaños cuando los modifiquemos:', X_flat_train.shape, X_flat_test.shape)

Tamaños originales: (70709, 35, 35, 3) (20624, 35, 35, 3) (70709, 63) (20624, 63)
Tamaños cuando los modifiquemos: (70709, 3675) (20624, 3675)


### Entrenamos el modelo

In [7]:
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(128, activation='relu', input_shape=X_flat_train[1].shape))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.05))
model.add(Dense(28, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(35, activation='relu'))
model.add(Dropout(0.05))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(63, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history_deep = model.fit(X_flat_train, y_train,
                          batch_size=128,
                          epochs=25,
                          verbose=1,
                          validation_data=(X_flat_test, y_test))
score = model.evaluate(X_flat_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               470528    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 28)                3612      
_________________________________________________________________
dropout_2 (Dropout)          (None, 28)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                1856      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

/home/sergio/.local/lib/python3.8/site-packages/keras/callbacks/callbacks.py:92: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.430911). Check your callbacks.
  warnings.warn(
/home/sergio/.local/lib/python3.8/site-packages/keras/callbacks/callbacks.py:92: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.215776). Check your callbacks.
  warnings.warn(


70709/70709 [==============================] - 8s 110us/step - loss: 0.6193 - accuracy: 0.7977 - val_loss: 0.5866 - val_accuracy: 0.8270
Epoch 7/25
70709/70709 [==============================] - 7s 95us/step - loss: 0.5434 - accuracy: 0.8243 - val_loss: 0.8191 - val_accuracy: 0.7626
Epoch 8/25
70709/70709 [==============================] - 6s 82us/step - loss: 0.4909 - accuracy: 0.8388 - val_loss: 0.5675 - val_accuracy: 0.8342
Epoch 9/25
70709/70709 [==============================] - 5s 75us/step - loss: 0.4489 - accuracy: 0.8548 - val_loss: 0.6736 - val_accuracy: 0.7900
Epoch 10/25
70709/70709 [==============================] - 7s 93us/step - loss: 0.4189 - accuracy: 0.8667 - val_loss: 0.4693 - val_accuracy: 0.8591
Epoch 11/25
70709/70709 [==============================] - 6s 81us/step - loss: 0.3804 - accuracy: 0.8786 - val_loss: 0.5388 - val_accuracy: 0.8481
Epoch 12/25
70709/70709 [==============================] - 5s 77us/step - loss: 0.3571 - accuracy: 0.8850 - val_loss: 0.4766 -

In [8]:
model.save('./Input/modelos/path_to_my_modelTesteado.h5')


### Creamos matriz de confusión

In [9]:
y = model.predict(X_flat_test)
d = confusion_matrix(np.argmax(y, axis=1),np.argmax(y_test, axis=1))
display(d)

array([[1764,    0,    0, ...,    0,    0,    0],
       [   0,  164,    0, ...,    0,    0,    0],
       [   0,    0,  309, ...,    0,    0,    0],
       ...,
       [   1,    0,    0, ...,  166,    0,    0],
       [   0,    0,    0, ...,    0, 1321,    0],
       [   0,    0,    0, ...,    0,    0,  249]])

In [10]:
print(np.unique(test_etiqueta))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62]


### Convertimosen un dataframe y lo preparamos para pintarlo

In [11]:
df_confusion = pd.DataFrame(d,columns=nombre_frutas_corregido)

ValueError: Shape of passed values is (63, 63), indices imply (63, 20624)

In [ ]:
display(df_confusion)

In [ ]:
corr_ma = df_confusion.corr()
corr_ma[corr_ma>0.8]


In [ ]:
f, ax = plt.subplots(figsize=(30, 20))

sns.heatmap(df_confusion,annot=True)